In [1]:
import tensorflow as tf

In [2]:
import pandas 

In [3]:
!mkdir Tensorflow

## **Setting up Tensorflow Object Detection API**

In [4]:
!git -C /content/Tensorflow  clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 43421, done.
remote: Total 43421 (delta 0), reused 0 (delta 0), pack-reused 43421
Receiving objects: 100% (43421/43421), 550.10 MiB | 30.46 MiB/s, done.
Resolving deltas: 100% (29535/29535), done.


In [5]:
%cd /content/Tensorflow/models/research
!protoc object_detection/protos/*.proto --python_out=.

/content/Tensorflow/models/research


In [6]:
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

Processing /content/Tensorflow/models/research
     |████████████████████████████████| 8.3MB 12.1MB/s 
     |████████████████████████████████| 358kB 37.3MB/s 
     |████████████████████████████████| 849kB 39.1MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 829kB 35.9MB/s 
     |████████████████████████████████| 1.4MB 40.7MB/s 
     |████████████████████████████████| 63.8MB 47kB/s 
     |████████████████████████████████| 153kB 43.5MB/s 
     |████████████████████████████████| 174kB 41.3MB/s 
     |████████████████████████████████| 1.1MB 38.2MB/s 
     |████████████████████████████████| 102kB 10.5MB/s 
     |████████████████████████████████| 36.6MB 69kB/s 
     |████████████████████████████████| 112kB 41.8MB/s 
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1568035 sha256=0b0f157d3916ea2

### **Testing TF2**

In [7]:
!python object_detection/builders/model_builder_tf2_test.py

2020-09-07 15:06:44.590942: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
2020-09-07 15:06:47.311984: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-09-07 15:06:47.356041: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-07 15:06:47.357054: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-09-07 15:06:47.357108: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] 

## **Preprocessing the Dataset**

In [8]:
# Created to keep all the working stuffs here
!mkdir /content/Tensorflow/workspace

In [9]:
!mkdir /content/Tensorflow/scripts

In [10]:
!mkdir /content/Tensorflow/scripts/preprocessing

In [11]:
!mkdir /content/Tensorflow/workspace/training_demo

In [12]:
# Creating directory inside training_demo
!mkdir /content/Tensorflow/workspace/training_demo/annotations
!mkdir /content/Tensorflow/workspace/training_demo/exported_models
!mkdir /content/Tensorflow/workspace/training_demo/pretrained_models
!mkdir /content/Tensorflow/workspace/training_demo/models


In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [15]:
!cp -R "/content/gdrive/My Drive/Deep Learning/Object Detection/Fire detection annotated dataset" "/content/Tensorflow/workspace/training_demo"

In [16]:
!cp "/content/gdrive/My Drive/Deep Learning/Object Detection/label_map.pbtxt" "/content/Tensorflow/workspace/training_demo/annotations"

In [17]:
!cp "/content/gdrive/My Drive/Deep Learning/Object Detection/generate_tfrecord.py" "/content/Tensorflow/scripts/preprocessing"

In [18]:
!mv "/content/Tensorflow/workspace/training_demo/Fire detection annotated dataset" "/content/Tensorflow/workspace/training_demo/images"

In [19]:
# Create train data:
!python3 /content/Tensorflow/scripts/preprocessing/generate_tfrecord.py -x /content/Tensorflow/workspace/training_demo/images/train -l /content/Tensorflow/workspace/training_demo/annotations/label_map.pbtxt -o /content/Tensorflow/workspace/training_demo/annotations/train.record

# Create test data:
!python3 /content/Tensorflow/scripts/preprocessing/generate_tfrecord.py -x /content/Tensorflow/workspace/training_demo/images/test -l /content/Tensorflow/workspace/training_demo/annotations/label_map.pbtxt -o /content/Tensorflow/workspace/training_demo/annotations/test.record

Successfully created the TFRecord file: /content/Tensorflow/workspace/training_demo/annotations/train.record
Successfully created the TFRecord file: /content/Tensorflow/workspace/training_demo/annotations/test.record


## **Now Training Part**

In [20]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz -P "/content/gdrive/My Drive/Deep Learning/Object Detection"

--2020-09-07 15:12:55--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.20.128, 2607:f8b0:400e:c04::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.20.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244817203 (233M) [application/x-tar]
Saving to: ‘/content/gdrive/My Drive/Deep Learning/Object Detection/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’

ssd_resnet50_v1_fpn 100%[===================>] 233.48M  64.6MB/s    in 3.6s    

2020-09-07 15:12:59 (64.9 MB/s) - ‘/content/gdrive/My Drive/Deep Learning/Object Detection/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’ saved [244817203/244817203]



In [21]:
!tar xvzf "/content/gdrive/My Drive/Deep Learning/Object Detection/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz" -C "/content/Tensorflow/workspace/training_demo/pretrained_models"


ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/saved_model.pb
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/assets/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/variables.index


In [22]:
!mkdir /content/Tensorflow/workspace/training_demo/models/my_ssd_resnet50

In [26]:
!cp "/content/Tensorflow/workspace/training_demo/pretrained_models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config" "/content/Tensorflow/workspace/training_demo/models/my_ssd_resnet50"

In [23]:
!cp "/content/Tensorflow/workspace/training_demo/pretrained_models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config" "/content/gdrive/My Drive/Deep Learning/Object Detection"

In [24]:
!cp "/content/Tensorflow/models/research/object_detection/model_main_tf2.py" "/content/Tensorflow/workspace/training_demo"

In [27]:
!python3 /content/Tensorflow/workspace/training_demo/model_main_tf2.py --model_dir=/content/Tensorflow/workspace/training_demo/models/my_ssd_resnet50 --pipeline_config_path=/content/Tensorflow/workspace/training_demo/models/my_ssd_resnet50/pipeline.config

2020-09-07 15:26:01.544838: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-07 15:26:04.031199: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-09-07 15:26:04.046673: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-07 15:26:04.047622: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-09-07 15:26:04.047671: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-07 15:26:04.050642: I tensorflow/stream_executor/pl

In [28]:
!cp "/content/Tensorflow/models/research/object_detection/exporter_main_v2.py" "/content/Tensorflow/workspace/training_demo"

In [32]:
%cd /content/Tensorflow/workspace/training_demo
!python3 ./exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/my_ssd_resnet50/pipeline.config --trained_checkpoint_dir ./models/my_ssd_resnet50/ --output_directory ./exported-models/my_model

/content/Tensorflow/workspace/training_demo
2020-09-07 15:41:21.921578: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-07 15:41:27.710976: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-09-07 15:41:27.761905: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-07 15:41:27.762866: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-09-07 15:41:27.762947: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-07 15:4

In [33]:
!cp -R "/content/Tensorflow/workspace/training_demo/exported-models/my_model" "/content/gdrive/My Drive/Deep Learning/Object Detection"